This is a script to merge census tracts and the isochrones generated from those tracts. It combines each isochrone and tract into a single polygon using their unique GEOID, then finds the number of grocery stores within each tract, and finally merges the store count data with the original tract shapefile.

In [4]:
import pandas as pd
import geopandas as gpd
import cenpy as cp
import numpy as np
import os
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import Point

In [5]:
def concat_int_cols(df, col1, col2, col3, fill1=2, fill2=3, fill3=6, return_col='GEOID'):
    """ Concatenate integer columns using zfill """
    df[[col1, col2, col3]] = df[[col1, col2, col3]].astype(str)
    df[col1] = df[col1].str.zfill(fill1)
    df[col2] = df[col2].str.zfill(fill2)
    df[col3] = df[col3].str.zfill(fill3)
    df[return_col] = df[[col1, col2, col3]].apply(lambda x: ''.join(x), axis=1)
    df[[col1, col2, col3, return_col]] = df[[col1, col2, col3, return_col]].astype(int)
    return df

In [6]:
# Import all shapefiles for merging isochrones and tracts
tracts = gpd.read_file(os.path.join('shapefiles', 'ti_2017_chi_tracts_simple.shp'))
isos = gpd.read_file(os.path.join('shapefiles', 'isochrones.shp'))
tracts = tracts[['GEOID', 'geometry']]
isos = isos[['GEOID', 'geometry']]

In [7]:
# Concatenate tracts and isochones
gdf = gpd.GeoDataFrame(pd.concat([tracts, isos], ignore_index=True))
gdf.index = gdf['GEOID']

In [8]:
# Merge the isochrone and tract polygons
fix = []
GEOID_list = gdf['GEOID'].unique()
for i, id in enumerate(GEOID_list):
    tmp = gdf[gdf['GEOID']==id].geometry
    sh = MultiPolygon([x.buffer(0) for x in tmp.geometry])
    fix.append({'GEOID': id, 'geometry':sh})
merged = gpd.GeoDataFrame(fix, columns=['GEOID', 'geometry'])
merged = merged.set_geometry('geometry')
merged['geometry'] = merged.geometry.buffer(0)
merged.crs = tracts.crs

In [15]:
# Create shapefile of points from the lat, long of grocery store queries
df = pd.read_csv(os.path.join('data', 'all_markets.csv'))

geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
points = gpd.GeoDataFrame(df, crs=merged.crs, geometry=geometry)
points.to_file(os.path.join('shapefiles', 'final_chi_points.shp'))

In [10]:
# Query census API to get tract level population
api_conn = cp.base.Connection('ACSSF5Y2015')
pop = api_conn.query(['B01001_001E'], geo_unit='tract:*', geo_filter = {'state':'17'})
pop.rename(
    columns={
        'B01001_001E': 'POP',
        'state': 'STATEFIP',
        'county': 'COUNTY',
        'tract': 'TRACT'
        },
        inplace=True)
pop = concat_int_cols(pop, 'STATEFIP', 'COUNTY', 'TRACT')
pop['GEOID'] = pop['GEOID'].astype(str)
print(pop)

       POP  STATEFIP  COUNTY   TRACT        GEOID
0     4558        17       1     100  17001000100
1     1900        17       1     201  17001000201
2     2666        17       1     202  17001000202
3     3399        17       1     400  17001000400
4     2415        17       1     500  17001000500
5     3759        17       1     600  17001000600
6     1243        17       1     700  17001000700
7     2929        17       1     800  17001000800
8     2603        17       1     900  17001000900
9     3737        17       1    1001  17001001001
10    3645        17       1    1002  17001001002
11    8076        17       1    1100  17001001100
12    4333        17       1   10100  17001010100
13    3524        17       1   10200  17001010200
14    6083        17       1   10300  17001010300
15    3219        17       1   10400  17001010400
16    3040        17       1   10500  17001010500
17    5952        17       1   10600  17001010600
18    2392        17       3  957600  17003957600


/anaconda3/envs/jupyterenv/lib/python3.6/site-packages/cenpy/remote.py:172: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df[cols] = df[cols].convert_objects(convert_numeric=convert_numeric)


In [11]:
# Merge the grocery store points with the merged tracts to determine counts, output final shapefile
counts = gpd.sjoin(merged, points, how='left', op='contains')
counts = pd.DataFrame(counts)
counts = counts.groupby('GEOID').size().reset_index(name='counts')
counts['GEOID'] = counts['GEOID'].astype(str)

In [12]:
# Merge neighborhood information onto each census tracts
tracts = gpd.read_file(os.path.join('shapefiles', 'ti_2017_chi_tracts_simple.shp'))
neighborhoods = gpd.read_file(os.path.join('shapefiles', 'ti_2012_chi_neighborhoods.shp')).to_crs(epsg = 4269)
tandh = gpd.sjoin(
    tracts.set_geometry(tracts.centroid),
    neighborhoods,
    how='left',
    op='within')
tandh = pd.DataFrame(tandh[['GEOID', 'PRI_NEIGH']])
tandh['GEOID'] = tandh['GEOID'].astype(str)

In [13]:
# Merge all the files together, calculate the stores per 1K population, then output to shapefile
tracts = gpd.read_file(os.path.join('shapefiles', 'ti_2017_chi_tracts_simple.shp'))
tracts = tracts\
  .merge(counts, on='GEOID')\
  .merge(pop, on='GEOID')\
  .merge(tandh, on='GEOID')
tracts['STORES_PER_1000'] = tracts['counts'] / tracts['POP'] * 1000
tracts = tracts.replace([np.inf, -np.inf], 0)

tracts = gpd.GeoDataFrame(tracts)
tracts.to_file(os.path.join('shapefiles', 'final_chi_tracts.shp'))